<a href="https://colab.research.google.com/github/gc2321/LangChain-Python/blob/main/3_2_LLMRouter_Transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

In [1]:
!pip install --upgrade --quiet langchain-openai langchain-community
!pip install -quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00

Usage:   
  pip3 install [op

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
open_ai_api ="/content/gdrive/MyDrive/llm/api_key.txt"
f = open(open_ai_api)
api_key = f.read()

In [5]:
os.environ['OPENAI_API_KEY'] = api_key

# LLM Router Chain

### Rout Template

In [6]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms.
You assume no prior knowledge. Here is the question\n{input}'''

In [7]:
expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a
PhD level understanding of Physics. Here is the question\n{input}'''

In [9]:
empty_template = 'empty'

### Route Prompts

In [10]:
prompt_infos = [
    {'name':'empty','description':'Replies to empty questions','prompt_template':empty_template},
    {'name':'advanced physics','description': 'Answers advanced physics questions',
     'prompt_template':expert_template},
    {'name':'beginner physics','description': 'Answers basic beginner physics questions',
     'prompt_template':beginner_template},

]

### Routing Chain Call

In [11]:
from langchain_openai import ChatOpenAI
from langchain.chains.router import MultiPromptChain
llm = ChatOpenAI()

In [12]:
chain = MultiPromptChain.from_prompts(llm, prompt_infos, verbose=True)

In [14]:
chain.invoke("How do magnets work?")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


{'input': 'How do magnets work?',
 'text': "Magnets are really cool because they have the ability to attract or repel other objects without even touching them! This is all thanks to the invisible magnetic field that surrounds the magnet. \n\nInside a magnet, there are tiny particles called electrons that all have their own magnetic field. When these particles line up in the same direction, they create a strong magnetic force. This force is what allows magnets to attract certain materials, like iron and steel.\n\nWhen two magnets are brought close together, their magnetic fields interact. If the poles of the magnets are opposite (one is north and the other is south), they will attract each other. But if the poles are the same (both north or both south), they will repel each other.\n\nSo basically, magnets work because of the magnetic fields created by the alignment of electrons inside them. It's like they have their own invisible superpower that allows them to interact with other object

In [15]:
chain.invoke("How do Feynman Diagrams work?")



> Entering new MultiPromptChain chain...
advanced physics: {'input': 'How do Feynman Diagrams work in quantum field theory?'}
> Finished chain.


{'input': 'How do Feynman Diagrams work in quantum field theory?',
 'text': 'Feynman diagrams are a graphical representation of the interactions between particles in quantum field theory. They were developed by physicist Richard Feynman in the 1940s as a way to visualize and calculate the probabilities of different particle interactions.\n\nIn a Feynman diagram, particles are represented by lines, with different types of particles denoted by different styles of lines. For example, fermions like electrons are represented by solid lines, while bosons like photons are represented by wavy lines. Vertices in the diagram represent interactions between particles, with each vertex corresponding to a specific interaction term in the quantum field theory Lagrangian.\n\nThe lines in the diagram also have arrows indicating the direction of particle flow, with outgoing particles on the right side of the diagram and incoming particles on the left side. The lines are also labeled with momentum and en

# Transform Chain

In [16]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [17]:
path = "/content/gdrive/MyDrive/llm/langchain/some_data/yelp_review.txt"

In [18]:
yelp_review = open(path).read()
print(yelp_review)

TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KN

In [21]:
# Custom Transformation Function
def transformer_fun(inputs: dict) -> dict:
    '''
    Notice how this always takes an inputs dictionary.
    Also outputs a dictionary. You can call the output and input keys whatever you want,
    just make sure to reference it correct in the chain call.
    '''
    # GRAB INCOMING CHAIN TEXT
    text = inputs['text']
    only_review_text = text.split('REVIEW:')[-1]
    lower_case_text = only_review_text.lower()
    return {'output':lower_case_text}

In [22]:
transform_chain = TransformChain(input_variables=['text'],
                                 output_variables=['output'],
                                 transform=transformer_fun)

In [23]:
transform_chain.invoke(yelp_review)

{'text': "TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!\n\nREVIEW:\nOH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!\n\nFIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.\n\nNOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!\n\nTHEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!\n\nTHE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIEN

In [24]:
template = "Create a one sentence summary of this review:\n{text}"

In [25]:
from langchain.prompts import ChatPromptTemplate

In [26]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(template)
summary_chain = prompt|llm

In [27]:
sequential_chain = transform_chain|summary_chain


In [28]:
result = sequential_chain.invoke(yelp_review)

In [29]:
result.content

'This review raves about the exceptional ambiance, delicious food, impeccable service, and heavenly desserts at a hidden gem of a restaurant, making it an absolute delight and a culinary haven.'